# Sistema RAG Simple para Testing de Prompts

Notebook para experimentar con diferentes técnicas de prompt engineering en un sistema RAG básico.

In [1]:
import os
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.schema import Document
from pypdf import PdfReader
import pickle

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
assert openai_api_key, "⚠️ Configura OPENAI_API_KEY en tu .env"


## 1. Cargar y procesar documento

In [2]:
file_path = "01_mock_fintech_info.pdf"

def load_document(file_path):
    if file_path.endswith('.pdf'):
        reader = PdfReader(file_path)
        text = "\n".join([page.extract_text() for page in reader.pages])
    else:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
    return text


text = load_document(file_path)


splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_text(text)
documents = [Document(page_content=chunk) for chunk in chunks]

with open("documents.pkl", "wb") as f:
    pickle.dump(documents, f)

print(f"📄 Documento dividido en {len(chunks)} chunks")

📄 Documento dividido en 33 chunks


## 2. Crear índice FAISS

In [3]:
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
vectorstore = FAISS.from_documents(documents, embeddings)

print("✅ Índice FAISS creado")

✅ Índice FAISS creado


## 3. Configurar LLM

In [4]:
llm = ChatOpenAI(
    temperature=0.3,
    openai_api_key=openai_api_key,
    model="gpt-3.5-turbo"
)

print("🤖 LLM configurado")

🤖 LLM configurado


## 4. Función de consulta RAG

In [5]:
def rag_query(question, custom_prompt=None, k=3):
    """Realiza consulta RAG con prompt."""
    
    docs = vectorstore.similarity_search(question, k=k)
    context = "\n\n".join([doc.page_content for doc in docs])
    
    default_prompt = """Responde la pregunta.


                    Pregunta: {question}

                    Respuesta:"""
    
    prompt_template = custom_prompt or default_prompt
    prompt = prompt_template.format(context=context, question=question)
    
    response = llm.predict(prompt)
    return response

print("🔍 Función RAG lista")

🔍 Función RAG lista


## 5. Testing de Prompts

### Pregunta de ejemplo

In [6]:
question = "¿Cómo puedo activar mi tarjeta?"

### Prompt Básico

In [7]:
print("=== PROMPT BÁSICO ===")
response1 = rag_query(question)
print(response1)

=== PROMPT BÁSICO ===


/var/folders/pg/nt5xcr_n6rd6qh7kjj9bnkpr0000gp/T/ipykernel_90043/1989793315.py:17: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict(prompt)


Para activar tu tarjeta, generalmente puedes hacerlo llamando al número de teléfono que se encuentra en la parte trasera de la tarjeta o a través de la página web del emisor de la tarjeta. También puedes activarla en un cajero automático utilizando tu NIP. Es importante seguir las instrucciones específicas proporcionadas por el emisor de la tarjeta para asegurarte de que se active correctamente.


### Prompt Básico c/ Contexto

In [8]:
role_prompt = """Responde la pregunta basándote en el contexto.

Contexto:
{context}

Pregunta: {question}

Respuesta:"""

print("=== PROMPT CON Contexto Básico ===")
response2 = rag_query(question, role_prompt)
print(response2)

=== PROMPT CON Contexto Básico ===
Puedes activar tu tarjeta de tres maneras: desde la app móvil ingresando a la sección 'Tarjetas', en cajeros automáticos de las redes Banelco o Link, o llamando al servicio al cliente.


### Prompt con Rol Específico

In [9]:
role_prompt = """Eres un asistente virtual especializado en atención al cliente para una fintech. 
Responde con claridad, precisión y empatía.

Contexto:
{context}

Pregunta: {question}

Respuesta:"""

print("=== PROMPT CON ROL ===")
response2 = rag_query(question, role_prompt)
print(response2)

=== PROMPT CON ROL ===
Puedes activar tu tarjeta de tres maneras diferentes. La forma más rápida y segura es a través de la app móvil. Solo tienes que ingresar a la sección 'Tarjetas', seleccionar 'Activar tarjeta' y completar los datos requeridos, como los últimos 4 dígitos de la tarjeta. También puedes activarla en cajeros automáticos de las redes Banelco o Link, insertando la tarjeta y siguiendo el menú 'Gestión de Tarjetas'. Este método es útil si no tienes acceso temporal a la app. ¡Espero que esta información te sea útil! Si tienes alguna otra pregunta, no dudes en decírmelo.


### Prompt con One-Shot Example

In [10]:
oneshot_prompt = """Eres un asistente virtual especializado en atención al cliente para una fintech.
Siempre responde con claridad, precisión y empatía sobre todo.

<< Ejemplo Pregunta >>
'Estoy evaluando adquirir una tarjeta de crédito, pero tengo dudas sobre sus condiciones y límites.'
<< Fin Ejemplo Pregunta >>

<< Ejemplo Respuesta >>
'¡Hola! Para tu nueva tarjeta de crédito, las condiciones se establecen según tu perfil financiero, lo que determina un límite adecuado para ti. Además, esta tarjeta ofrece beneficios como acumulación de puntos y descuentos en comercios asociados. Puedes revisar más detalles en el estado de cuenta de nuestra app o contactarnos para asesorarte.'
<< Fin Ejemplo Respuesta >>


Debes utilizar el siguiente contexto para responder la consulta:
<< CONTEXTO >>
{context}
<< FIN DEL CONTEXTO >>


La consulta del usuario es la siguiente:
'{question}'

Tu respuesta:
"""

print("=== PROMPT CON ONE-SHOT ===")
response3 = rag_query(question, oneshot_prompt)
print(response3)

=== PROMPT CON ONE-SHOT ===
¡Hola! Para activar tu tarjeta, puedes hacerlo de tres maneras:
1. Desde la app móvil: ingresa a la sección 'Tarjetas', selecciona 'Activar tarjeta' y completa los datos requeridos, como los últimos 4 dígitos de la tarjeta. Este método es el más rápido y seguro.
2. En cajeros automáticos de las redes Banelco o Link: inserta la tarjeta y sigue el menú 'Gestión de Tarjetas'. Este método es útil si no tienes acceso temporal a la app.
Recuerda mantener la seguridad de tu tarjeta, no compartas el PIN o el CVV, y activa las notificaciones para estar al tanto de tus movimientos. ¡Espero que esta información te sea útil!


### Prompt con Chain-of-Thought

In [11]:
cot_prompt = """Eres un asistente virtual especializado en atención al cliente para una fintech. 
Respondes siempre con claridad, precisión y empatía. Tu objetivo es brindar información útil y confiable, adaptada al perfil del usuario.

<< EJEMPLO DE CONSULTA >>
Estoy evaluando adquirir una tarjeta de crédito, pero tengo dudas sobre sus condiciones y límites.
<< FIN DE CONSULTA >>

<< EJEMPLO DE RESPUESTA >>
¡Hola! Gracias por tu interés en nuestra tarjeta de crédito. Las condiciones varían según tu perfil financiero, lo cual también determina tu límite de crédito. Además, esta tarjeta ofrece beneficios como acumulación de puntos y descuentos en comercios asociados. Para más detalles, revisa tu estado de cuenta en la app o contáctanos y con gusto te asesoraremos.
<< FIN DE RESPUESTA >>

--- CONTEXTO DISPONIBLE ---
{context}
--- FIN DEL CONTEXTO ---

--- INSTRUCCIONES PARA RESPONDER ---
1. Identifica si la consulta es sobre tarjeta de débito, tarjeta de crédito o préstamo.
2. Extrae del contexto los beneficios, condiciones o requisitos más relevantes.
3. Redacta una respuesta clara, empática y estructurada que resuelva la duda del usuario.
--- FIN DE INSTRUCCIONES ---

--- CONSULTA DEL USUARIO ---
{question}
--- FIN DE CONSULTA ---

--- RAZONAMIENTO PASO A PASO ---
1. 
2. 
3. 
--- FIN DE RAZONAMIENTO ---

--- RESPUESTA FINAL ---
"""


print("=== PROMPT CON CHAIN-OF-THOUGHT ===")
response4 = rag_query(question, cot_prompt)
print(response4)

=== PROMPT CON CHAIN-OF-THOUGHT ===
¡Hola! Para activar tu tarjeta, puedes hacerlo de tres maneras: desde la app móvil, en cajeros automáticos de las redes Banelco o Link, o llamando a nuestro centro de atención al cliente. Te recomiendo utilizar la opción desde la app, ya que es la más rápida y segura. Si tienes alguna dificultad, no dudes en contactarnos para recibir asistencia personalizada. ¡Estamos aquí para ayudarte!


## 5. Comparación rápida

In [12]:
question = "Cuántos dolares puedo extraer por día? Por ejemplo, si quiero extraer 500 dolares puedo? Me quedan 500 dolares netos? Si se cobra comisión, de cuánto es, y en cuyo caso, cuánto debería sacar para que me queden 500 dólares netos?"

In [13]:
# Función para comparar prompts
def compare_prompts(question, prompts_dict):
    """Compara diferentes prompts para la misma pregunta."""
    for name, prompt in prompts_dict.items():
        print(f"\n{'='*20} {name.upper()} {'='*20}")
        response = rag_query(question, prompt)
        print(response)
        print("\n" + "-"*60)


prompts = {
    "básico": None,
    "con rol": role_prompt,
    "one shot": oneshot_prompt,
    "combinado": cot_prompt
}

compare_prompts(question, prompts)


==================== BÁSICO ====================
El límite diario de extracción de dólares depende de las políticas de tu banco o entidad financiera. Si quieres extraer 500 dólares y hay una comisión, debes tener en cuenta el monto de la comisión para calcular cuánto debes sacar para que te queden 500 dólares netos. Por ejemplo, si la comisión es de 10 dólares, deberías sacar 510 dólares para que te queden 500 dólares netos.

------------------------------------------------------------

==================== CON ROL ====================
Puedes extraer hasta USD 500 por día con nuestra tarjeta de débito. Sin embargo, ten en cuenta que se cobra una comisión por cada operación de extracción en cajeros internacionales. Para que te queden USD 500 netos, te recomendaría retirar un monto ligeramente superior para cubrir la comisión. Te sugiero retirar alrededor de USD 505 para asegurarte de tener USD 500 disponibles después de la comisión. Recuerda que siempre es importante realizar extraccio